In [1]:
from IPython.display import display
from copy import deepcopy
import numpy as np
import pandas as pd
import pyvinecopulib as pv

def make_triangular_array(d):
    pair_copulas = np.empty(shape=(d-1,), dtype='object')
    for j in range(d-1)[::-1]:
        pair_copulas[j] = list(np.empty(shape=(d-j-1,), dtype='object'))
    return list(pair_copulas)

In [4]:
n = 1000
d = 4
p = 0.2

structure = pv.RVineStructure.simulate(d)

pair_copulas = []
for j in range(d - 1):
    tmp = []
    pair_copulas.append(tmp)
    for _ in range(d - j - 1):
        rho = np.minimum(np.maximum(np.random.beta(1, 0.75), 0.01), 0.99)
        tmp.append(pv.Bicop(family=pv.BicopFamily.gaussian, parameters=[[rho]]))

cop = pv.Vinecop(structure, pair_copulas)

U = cop1.simulate(n=n, seeds=list(1 + np.arange(d)))

In [5]:
cop

<pyvinecopulib.Vinecop>
** Tree: 0
2,1 <-> Gaussian, parameters = 0.636167
3,1 <-> Gaussian, parameters = 0.672982
4,1 <-> Gaussian, parameters = 0.397528
** Tree: 1
2,4 | 1 <-> Gaussian, parameters = 0.99
3,4 | 1 <-> Gaussian, parameters = 0.195834
** Tree: 2
2,3 | 4,1 <-> Gaussian, parameters = 0.713836

In [3]:
U

array([[0.72326351, 0.78080499, 0.69804208, 0.75101924],
       [0.67571195, 0.3048536 , 0.79207663, 0.63710412],
       [0.48989027, 0.16343166, 0.75955506, 0.43017745],
       ...,
       [0.90368168, 0.36505584, 0.98653356, 0.94743454],
       [0.27035184, 0.80870488, 0.89054095, 0.88412868],
       [0.06912669, 0.98952835, 0.17611949, 0.24519554]])

In [6]:
cop.matrix

array([[1, 1, 1, 1],
       [4, 4, 4, 0],
       [3, 3, 0, 0],
       [2, 0, 0, 0]], dtype=uint64)

In [7]:
var_mis = 1

In [95]:
T = cop.matrix

In [15]:
for j in range(d):
    for i in range(d-j-1):
        print(i,j, cop.order[j])
        

0 0 2
1 0 2
2 0 2
0 1 3
1 1 3
0 2 4


In [44]:
def proc0(T_in, var_mis):
    d = T_in.shape[0]
    T_out = deepcopy(T_in)
    for j in range(d):
        if var_mis in T_out[:d-j-1,j]:
            T_out[:, j] = 0
    return T_out

In [118]:
def proc1(T_in, var_mis):
    d = T_in.shape[0]
    T_out = deepcopy(T_in)
    order = [T[d-j-1, j] for j in range(d)]
    for j in range(d):
        mis_idx, = np.where(T_out[:, j] == var_mis)
        if len(mis_idx) == 1:
            var_deleted = T_out[(mis_idx[0]+1):d-j-1, j]
            for var in var_deleted:
                T_out[:, order.index(var)] = 0
            T_out[(mis_idx[0]+1):d-j-1, j] = 0
    return T_out

In [24]:
# store pair copulas

True